In [ ]:
import math
import re

In [ ]:
CPU="M3_dit"

In [ ]:
amx_polymul = {}
amx_polymodmul = {}
amx_polymodmul_karatsuba = {}

# Read polynomial multiplication cycle counts from benchmark results
with open(f"../speed_results_{CPU}/polymul_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymul (\d+) x (\d+): (\d+)", line)
        if match:
            amx_polymul[int(match.group(1))] = int(match.group(3))

with open(f"../speed_results_{CPU}/polymodmul_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymodmul (\d+) x (\d+): (\d+)", line)
        if match:
            amx_polymodmul[int(match.group(1))] = int(match.group(3))

with open(f"../speed_results_{CPU}/polymodmul_karatsuba_scaling.txt", "r", encoding="utf-8") as f:
    for line in f:
        match = re.match(r"amx_polymodmul (\d+) x (\d+): (\d+)", line)
        if match:
            amx_polymodmul_karatsuba[int(match.group(1))] = int(match.group(3))


In [ ]:
# The algorithm is based on dynamic programming. min_costs stores the minimum cost for each size, as well as the
# algorithm used to recurse (or "schoolbook" if this is the base case) to achieve this minimum cost.
min_costs = {}
min_costs[32] = ["schoolbook", amx_polymul[32]]

In [ ]:
# The main function for calculating a lower bound on the minimum cost for n-coefficient polynomial multiplication.
# It employs a dynamic programming algorithm, which computes the minimum cost among the possible recursion strategies
# (schoolbook, Karatsuba, Toom-3 and Toom-4) for each level.
def min_cost(n):
    if n in min_costs:
        return min_costs[n][1]
    else:
        cost_schoolbook = ["schoolbook", amx_polymul[n]]
        cost_karatsuba = ["karatsuba", karatsuba(n)]
        cost_tc3 = ["tc3", tc3(n)]
        cost_tc4 = ["tc4", tc4(n)]
        min_costs[n] = min(
            cost_schoolbook, cost_karatsuba, cost_tc3, cost_tc4, key = lambda x: x[1]
        )
        return min_costs[n][1]

In [ ]:
# Karatsuba cost computation
def karatsuba(n):
    # Impossible to apply Karatsuba if n < 64
    if n < 64:
        return math.inf

    n32 = n // 32
    n2 = n32 // 2

    # If the number of blocks is even, we split the polynomial in two equal halves
    if n32 % 2 == 0:
        return 3 * min_cost(32 * n2)
    # Otherwise, one of the blocks will be larger than the other. Consider the product f(x)*g(x), such that
    # f(x) = f1(x)*x^(n2 + 1) + f0(x) and g(x) = g1(x)*x^n2 + g0(x), so that f0(x) and g0(x) are degree-32*(n2 + 1)
    # polynomials, while f1(x) and g1(x) are degree-32*n2 polynomials. Thus, we have that f1(x) + f0(x) and 
    # g1(x) + g0(x) are also degree-32*(n2 + 1) polynomials. As such, of the three multiplications required, two
    # (f0(x)*g0(x) and (f1(x) + f0(x))*(g1(x) + g0(x))) have larger input polynomials of degree 32*(n2 + 1), while
    # f1(x)*g1(x) has smaller input polynomials of degree 32*n2.
    else:
        return 2 * min_cost(32 * (n2 + 1)) + min_cost(32 * n2)


In [ ]:
# Toom-3 cost computation
def tc3(n):
    # Impossible to apply Toom-3 if n < 96
    if n < 96:
        return math.inf

    n32 = n // 32
    n3 = n32 // 3

    # If the number of blocks is divisible by 3, we split the polynomial in three equal parts
    if n32 % 3 == 0:
        return 5 * min_cost(32 * n3)
    # If the number of blocks is 1 modulo 3, we have two blocks of size n3 and one of size n3 + 1. Let
    # f(x) = f2(x)*x^(2*n3 + 1) + f1(x)*x^n3 + f0(x) and similarly for g(x). Thus, f2(x), g2(x), f0(x) and g0(x) are
    # degree-32*n3 polynomials, while f1(x) and g1(x) are degree-32*(n3 + 1) polynomials. The evaluations at 0 and
    # infinity call for computing f0(x)*g0(x) and f2(x)*g2(x), whose input polynomials are of degree 32*n3, while the
    # remaining 3 evaluations employ larger input polynomials of degree 32*(n3 + 1).
    elif n32 % 3 == 1:
        return 2 * min_cost(32 * n3) + 3 * min_cost(32 * (n3 + 1))
    # If the number of blocks is 2 modulo 3, we have one block of size n3 and two of size n3 + 1. We can assign the
    # smaller polynomial to either f0(x) and g0(x), in which case the evaluation at 0 can use smaller input polynomials
    # of degree 32*n3, or to f2(x) and g2(x), in which case the evaluation at infinity can use these smaller input
    # polynomials. Either way, the remaining 4 evaluations must employ larger input polynomials of degree 32*(n3 + 1).
    else:
        return 1 * min_cost(32 * n3) + 4 * min_cost(32 * (n3 + 1))

In [ ]:
# Toom-4 cost computation
def tc4(n):
    # Impossible to apply Toom-4 if n < 128
    if n < 128:
        return math.inf

    n32 = n // 32
    n4 = n32 // 4

    # If the number of blocks is divisible by 4, we split the polynomial in four equal parts
    if n32 % 4 == 0:
        return 7 * min_cost(32 * n4)
    # If the number of blocks is 1 or 2 modulo 4, we have at least two blocks of size n4. Similarly to the Toom-3 case,
    # these can be used for evaluation at 0 and infinity, using smaller input polynomials of degree 32*n4. The remaining
    # 5 evaluations employ larger input polynomials of degree 32*(n4 + 1).
    if n32 % 4 == 1 or n32 % 4 == 2:
        return 2 * min_cost(32 * n4) + 5 * min_cost(32 * (n4 + 1))
    # If the number of blocks is 3 modulo 4, we have one block of size n4 and three of size n4 + 1. Similarly to the
    # Toom-3 case, only one small pair of input polynomials can be used for evaluation, at either 0 or infinity. The
    # remaining 6 evaluations employ larger input polynomials of degree 32*(n4 + 1).
    else:
        return 1 * min_cost(32 * n4) + 6 * min_cost(32 * (n4 + 1))

In [ ]:
# Computes and prints the minimum cost for all NTRU parameter sets, rounded up to the nearest multiple of 32;
# i.e., 512 for n = 509, 704 for n = 677 and 701, and 832 for n = 821.
for n in [512, 704, 832]:
    print(
        f"n = {n} schoolbook performance = {amx_polymodmul[n]} subquadratic lower bound = {min_cost(n)} " +
        f"1L Karatsuba lower bound = {3*amx_polymul[n // 2]} 1L Karatsuba performance = {amx_polymodmul_karatsuba[n]}"
    )

In [ ]:
# Prints the memoized evaluations, helping to visualize the recursive strategies
for key, value in sorted(min_costs.items()):
    print("{}: {}".format(key, value))